In [48]:
with open("AeCa.txt",'r',encoding='UTF-8') as f:
    text=f.read()

In [49]:
print("Length of the dataset of characters: ",len(text))                                          #Total Length of the text file

Length of the dataset of characters:  1591049


In [50]:
chars=sorted(list(set(text)))                                #Total number of different characters in the DNA sequence
vocab_size=len(chars)
print(chars,vocab_size)

['A', 'C', 'G', 'T'] 4


In [51]:
stoi={s:i for i,s in enumerate(chars)}
itos={i:s for i,s in enumerate(stoi)}
stoi,itos

({'A': 0, 'C': 1, 'G': 2, 'T': 3}, {0: 'A', 1: 'C', 2: 'G', 3: 'T'})

In [52]:
encode=lambda word:[stoi[s] for s in word]
decode=lambda num: ''.join([itos[i] for i in num])

d_en=encode("GTTA")
d_de=decode(d_en)
print(d_en,d_de)

[2, 3, 3, 0] GTTA


In [53]:
import torch

In [54]:
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)

torch.Size([1591049]) torch.int64


In [55]:
l=int(0.8*data.shape[0])
h=int(0.9*data.shape[0])
train_data=data[:l]
val_data=data[l:h]
test_data=data[h:]
print(train_data.shape,val_data.shape,test_data.shape)

torch.Size([1272839]) torch.Size([159105]) torch.Size([159105])


In [56]:
block_size=8
train_data[:block_size+1]

tensor([2, 1, 1, 2, 1, 1, 1, 1, 1])

In [57]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range (block_size):
    context=x[:t+1]
    target=y[t]
    print(f"In the context of {context} the target is {target}")

In the context of tensor([2]) the target is 1
In the context of tensor([2, 1]) the target is 1
In the context of tensor([2, 1, 1]) the target is 2
In the context of tensor([2, 1, 1, 2]) the target is 1
In the context of tensor([2, 1, 1, 2, 1]) the target is 1
In the context of tensor([2, 1, 1, 2, 1, 1]) the target is 1
In the context of tensor([2, 1, 1, 2, 1, 1, 1]) the target is 1
In the context of tensor([2, 1, 1, 2, 1, 1, 1, 1]) the target is 1


In [58]:
import torch.nn as nn

In [59]:
class Linear(nn.Module):
    def __init__(self,fan_in,fan_out,bias=False):
        super().__init__()
        self.W=torch.randn((fan_out,fan_in))
        self.b=torch.randn(fan_out) if bias==True else 0
    
    def forward(self, x):
        #print(self.W.shape)
        return x @ self.W.T + self.b


In [61]:
q=Linear(3,4)
x=torch.randn((4,3))
print(x.shape)
q(x)


torch.Size([4, 3])
torch.Size([4, 3])


tensor([[-0.6594, -0.3398,  0.5017, -0.5270],
        [ 1.1055, -0.0576, -0.2546,  0.3698],
        [ 1.3453,  1.1977, -1.0826,  3.2578],
        [ 2.4306,  0.1036, -0.3579,  2.7908]])